# Inteligência Artificial - Exercício Árvores

Implemente o A\* para resolver o quebra-cabeças abaixo com duas heurísticas distintas: 
    
    - h1(n) = número de pedras fora do lugar
    - h2(n) = distância total à la Manhattan (horizontal ou vertical) = somatório do deslocamento de cada peça até a posição final

Entregar o código fonte e respostas às seguintes questões (ambos no moodle):
    - Qual o tamanho do espaço de estados?
    - Para as duas heurísticas, justificar se são admissíveis e consistentes.
    - Compare as soluções encontradas com h1(n) e h2(n) em termos de:
        * solução encontrada;
        * tempo: número de nós criados em tempo de busca;
        * espaço: número máximo de nós armazenados em memória em tempo de busca.

![puzzle](res/a_estrela_puzzle.png)


As heurísticas solicitadas para resolução desse problema foram: Distância Manhattan e Peças fora do lugar. Sendo assim, definiu-se duas funções para retornar um valor referente a essas heurísticas(dados dois estados para comparação)

In [1]:
from scipy.spatial import distance
import time
MANHATTAN = 1
PIECES = 2

# funcao auxiliar para retornar posicao no jogo para o calculo da distancia manhattan
def index2d(index):
    return int(index / 3), index % 3

#calcula o somatorio da distancia manhatan de todas as peças(incluido a casa vazia)
def manhattan(matrix1, matrix2):
    dist = 0
    for i in matrix1:
        dist += distance.cityblock(index2d(matrix1.index(i)),index2d(matrix2.index(i)))
    return dist

#calcula o somatorio de peças fora do ligar(incluido a casa vazia)
def pieces(matrix1,matrix2):
    pieces = 0
    for i in matrix1:
        if matrix1.index(i) != matrix2.index(i):
            pieces += 1
    return pieces

def heuristic(matrix1, matrix2, heuristic):
    if heuristic == MANHATTAN:
        return manhattan(matrix1, matrix2)
    elif heuristic == PIECES:
        return pieces(matrix1, matrix2)

Para facilitar o algoritmo A\* vamos criar uma classe para mapear o estado do nosso problema. Todas as informações que a gente precisa soobre o estado estarão nos parâmetros.

In [2]:

class State:
    def __init__(self, configuration, actual_cost, heuristic_cost, came_from):
        self.configuration = configuration
        self.actual_cost = actual_cost
        self.heuristic_cost = heuristic_cost
        self.total_cost = self.actual_cost + self.heuristic_cost
        self.came_from = came_from
    def __cmp__(self,other):
        return (self.total_cost > other.total_cost) - (self.total_cost < other.total_cost)

No Algoritmo do agente será necessário a função que gerará os filhos do estado atual. O conceito básico dessa função será encontrar a posição da casa vazia e gerar estados mudando as peças vizinhas.

In [3]:
def generate_children(state,final_configuration,method):
    # lista com os filhos
    children = []

    # encontrando a posicao da casa vazia e as casas vizinhas
    empty_pos = state.configuration.index(0)
    above_pos = empty_pos - 3
    below_pos = empty_pos + 3
    right_pos = empty_pos + 1
    left_pos = empty_pos - 1

    # trocando com a posicao ACIMA caso existir
    if above_pos >= 0:
        new_conf = state.configuration.copy()
        new_conf[empty_pos] = new_conf[above_pos]
        new_conf[above_pos] = 0
        child = State(new_conf, state.actual_cost + 1, heuristic(new_conf, final_configuration, method), state)
        children.append(child)

    # trocando com a posicao ABAIXO caso existir
    if below_pos < len(state.configuration):
        new_conf = state.configuration.copy()
        new_conf[empty_pos] = new_conf[below_pos]
        new_conf[below_pos] = 0
        child = State(new_conf, state.actual_cost + 1, heuristic(new_conf, final_configuration, method), state)
        children.append(child)

    # trocando com a posicao DIREITA
    if right_pos < len(state.configuration):
        new_conf = state.configuration.copy()
        new_conf[empty_pos] = new_conf[right_pos]
        new_conf[right_pos] = 0
        child = State(new_conf, state.actual_cost + 1, heuristic(new_conf, final_configuration, method), state)
        children.append(child)

    # trocando com a posicao ESQUERDA caso existir
    if left_pos >= 0:
        new_conf = state.configuration.copy()
        new_conf[empty_pos] = new_conf[left_pos]
        new_conf[left_pos] = 0
        child = State(new_conf, state.actual_cost + 1, heuristic(new_conf, final_configuration, method), state)
        children.append(child)
    return children

Também será necessário criar uma função para controlar a adição de elementos na fronteira que, se houver uma configuração igual, só mantem o estado com o melhor custo total

In [4]:
#função para adicionar na fronteira e que verifica se não há um elemento melhor
def add_to_border(state_list, border):
    already_on_border = False
    for state in state_list:
        for border_state in border:
            if state.configuration == border_state.configuration:
                already_on_border = True
                if state.actual_cost < border_state.actual_cost:
                    border[border.index(border_state)] = state
        if not already_on_border:
            border.append(state)

Com a estrutura básica criada, agora é necessário modelar o agente que irá fazer a busca A\*. A ideia básica do agente é:
1. adiciona estado na fronteira
2. enquanto fronteira != de vazio
    - seleciona o melhor da fronteira removendo ele da fronteira e adicionando aos "estados visitados"
    - se for estado final, adiciona aos estados e percorre o caminho came_from --> fim
    - se nao for, gera os estados filhos adicionando na fronteira
    - volta 2

In [5]:
def algorithm(initial_state, final_state, heuristic_method):
    start_time = time.time()
    generation = 0 #geracoes de filhos
    method = heuristic_method #metodo heuristico desejado
    created_nodes = 0 #quantidade de nós criados
    max_stored_nodes = 0 #quantidade de nós guardados na memoria
    border = [] #fronteira
    visited = [] #nós visitados
    result = [] #resutado do algoritmo
    found_solution = False # controle solucao
    
    # representando os estados
    si = initial_state
    sf = final_state

    # Criando o estado inicial com todos os valores
    start_state = State(si, 0, heuristic(si, sf, method), None)

    # adicionando o estado inicial à fronteira
    border.append(start_state)

    # enquanto a fronteira nao estiver vazia
    while len(border) > 0:
        generation += 1

        # ordenando para poder obter o melhor a cada iteração
        border.sort(key=lambda state: state.total_cost)

        # selecionando o melhor e colocando na lista dos nós visitados
        actual_state = border[0]
        visited.append(actual_state)

        # removendo o nó visitado
        del (border[0])

        # verificando se o estado já é o final
        if actual_state.configuration == sf:
            found_solution = True
            break
        else:
            # gerando os estados filhos
            children = generate_children(actual_state, sf, method)
            created_nodes += len(children)
            
            #adicionado à fronteira
            add_to_border(children, border)
            if len(border) > max_stored_nodes:
                max_stored_nodes = len(border)
    
    if found_solution:
        state = visited.pop()
        
        while state is not start_state:
            result.append(state)
            state = state.came_from

        #incuindo o no inicial ao resultado
        result.append(start_state)
        #inverntendo a ordem para imprimir o passo correto
        result.reverse()
    else:
        print('Nao encontrou solucao')
    
    end_time = time.time()
    total_time = end_time - start_time
    
    return(result, total_time, generation, created_nodes, max_stored_nodes)

Aqui rodamos o algoritmo para cada uma das heuristicas

In [6]:
initial_state = [7, 2, 4,
                 5, 0, 6,
                 8, 3, 1]

final_state = [0, 1, 2,
               3, 4, 5,
               6, 7, 8]
  
result_manhattan = algorithm(initial_state, final_state, MANHATTAN)
result_pieces = algorithm(initial_state, final_state, PIECES)

Imprimindo os resultados

In [7]:
def print_solution(name,result):
    print('RESULTADO - ' + str(name))
    print('Tempo total: ' + str(result[1]) + ' segundos')
    print('Gerações: ' + str(result[2]))
    print('Profundidade da solução: ' + str(len(result[0])) + ' passos')
    print('Nós criados em tempo de buscas: ' + str(result[3]))
    print('Máximo de nós armazenados: ' + str(result[4]))
    print('Solução:' + '\n')
    for r in result[0]:
        print('{s[0]} {s[1]} {s[2]}\n{s[3]} {s[4]} {s[5]}\n{s[6]} {s[7]} {s[8]} \n------'.format(s=r.configuration))
    print('\n==============================================\n')

### MANHATTAN

In [8]:
print_solution('MANHATTAN', result_manhattan)

RESULTADO - MANHATTAN
Tempo total: 179.6111798286438 segundos
Gerações: 31093
Profundidade da solução: 29 passos
Nós criados em tempo de buscas: 95851
Máximo de nós armazenados: 7486
Solução:

7 2 4
5 0 6
8 3 1 
------
7 2 4
5 6 0
8 3 1 
------
7 2 4
5 6 8
0 3 1 
------
7 2 4
5 6 8
3 0 1 
------
7 2 4
5 6 8
3 1 0 
------
7 2 4
5 6 0
3 1 8 
------
7 2 4
5 6 3
0 1 8 
------
7 2 4
0 6 3
5 1 8 
------
0 2 4
7 6 3
5 1 8 
------
2 0 4
7 6 3
5 1 8 
------
2 4 0
7 6 3
5 1 8 
------
2 4 3
7 6 0
5 1 8 
------
2 4 3
7 6 8
5 1 0 
------
2 4 3
7 6 0
5 1 8 
------
2 4 3
7 6 5
0 1 8 
------
2 4 3
7 6 5
1 0 8 
------
2 4 3
7 6 5
0 1 8 
------
2 4 3
0 6 5
7 1 8 
------
2 4 3
6 0 5
7 1 8 
------
2 4 3
6 1 5
7 0 8 
------
2 4 3
6 1 5
0 7 8 
------
2 4 3
0 1 5
6 7 8 
------
2 4 3
1 0 5
6 7 8 
------
2 0 3
1 4 5
6 7 8 
------
0 2 3
1 4 5
6 7 8 
------
1 2 3
0 4 5
6 7 8 
------
1 2 0
3 4 5
6 7 8 
------
1 0 2
3 4 5
6 7 8 
------
0 1 2
3 4 5
6 7 8 
------




### PEÇAS FORA DO LUGAR

In [9]:
print_solution('PEÇAS FORA DO LUGAR', result_pieces)

RESULTADO - PEÇAS FORA DO LUGAR
Tempo total: 1265.8206856250763 segundos
Gerações: 102316
Profundidade da solução: 35 passos
Nós criados em tempo de buscas: 325616
Máximo de nós armazenados: 24978
Solução:

7 2 4
5 0 6
8 3 1 
------
7 2 4
5 6 0
8 3 1 
------
7 2 4
5 6 8
0 3 1 
------
7 2 4
5 6 8
3 0 1 
------
7 2 4
5 6 8
3 1 0 
------
7 2 4
5 6 0
3 1 8 
------
7 2 4
5 6 3
0 1 8 
------
7 2 4
0 6 3
5 1 8 
------
0 2 4
7 6 3
5 1 8 
------
2 0 4
7 6 3
5 1 8 
------
2 4 0
7 6 3
5 1 8 
------
2 4 7
0 6 3
5 1 8 
------
2 4 7
6 0 3
5 1 8 
------
2 4 7
6 3 0
5 1 8 
------
2 4 0
6 3 7
5 1 8 
------
2 0 4
6 3 7
5 1 8 
------
2 3 4
6 0 7
5 1 8 
------
2 3 4
6 7 0
5 1 8 
------
2 3 4
6 7 5
0 1 8 
------
2 3 4
6 7 5
1 0 8 
------
2 3 4
6 0 5
1 7 8 
------
2 3 4
6 5 0
1 7 8 
------
2 3 4
6 5 1
0 7 8 
------
2 3 4
0 5 1
6 7 8 
------
0 3 4
2 5 1
6 7 8 
------
3 0 4
2 5 1
6 7 8 
------
3 4 0
2 5 1
6 7 8 
------
3 4 2
0 5 1
6 7 8 
------
3 4 0
2 5 1
6 7 8 
------
3 4 1
2 5 0
6 7 8 
------
3 4 1
2 0 5
6